In [1]:
import pandas as pd
import nltk
import re, random, os
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [2]:
df = pd.read_csv('Oneplus_reviews.csv')

In [3]:
df.shape

(300, 5)

In [4]:
df.head(10)

,Unnamed: 0,Model,rating,title,review_text
0,0,Oneplus 11,5.0 out of 5 stars,5.0 out of 5 stars\nBeast (best) phone at this...,\nBeen using OnePlus 11 for about a week now a...
1,1,Oneplus 11,5.0 out of 5 stars,5.0 out of 5 stars\nGood phone and definite fl...,\nI bought this after lots of research. Green ...
2,2,Oneplus 11,4.0 out of 5 stars,4.0 out of 5 stars\nBeast\n,\nAs a OnePlus family member I just upgraded f...
3,3,Oneplus 11,5.0 out of 5 stars,"5.0 out of 5 stars\nits not a bad phone, its n...","\nso before i start, I've been using Samsung p..."
4,4,Oneplus 11,5.0 out of 5 stars,5.0 out of 5 stars\nPretty good\n,\nOperating system -Coming to OxygenOS from MI...
5,5,Oneplus 11,1.0 out of 5 stars,1.0 out of 5 stars\nWHAT A SHOCKER OF A LET DO...,"\nIn the family, we had several OnePlus device..."
6,6,Oneplus 11,5.0 out of 5 stars,5.0 out of 5 stars\nCan go toe to toe with the...,\nI'm writing this review after 7 months of us...
7,7,Oneplus 11,5.0 out of 5 stars,5.0 out of 5 stars\nSo far so good\n,\nIt's second day and it looks very satisfying...
8,8,Oneplus 11,4.0 out of 5 stars,4.0 out of 5 stars\nLove the Phone and it's qu...,"\nEverything is great, don't waste money going..."
9,9,Oneplus 11,4.0 out of 5 stars,4.0 out of 5 stars\nPowerpacked device and gre...,\nI bought oneplus11 256GB variant black colou...


In [5]:
import re

def remove_ratings(title):
    return re.sub(r'\d+\.\d+ out of \d+ stars', '', title).strip()

In [6]:
df['title'] = df['title'].apply(remove_ratings)

In [7]:
df['reviews'] =df['review_text']

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohdzahan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mohdzahan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE
)

In [11]:

df['reviews'] = df['reviews'].astype(str)

In [12]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    text = text.replace('\n', '.')
    text = emoji_pattern.sub(r'', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()  
    text = ' '.join(word for word in text.split() if word not in stop_words)  
    return text

df['reviews'] = df['reviews'].apply(preprocess)

In [13]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 1.7 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [14]:
nlp = spacy.load('en_core_web_md', disable = ['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    output = []
    for text in texts:
        doc = nlp(text)
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

tokenized_reviews = lemmatization(df['reviews'].tolist())

In [ ]:
def remove_topic(tokenized_reviews):
    unwanted = ['phone', 'oneplus','mobile']
    for i in unwanted:
        while i in tokenized_reviews:
            tokenized_reviews.remove(i)
    return tokenized_reviews

tokenized_reviews = remove_topic(tokenized_reviews)
tokenized_reviews

In [15]:
print(tokenized_reviews)

[['oneplus', 'week', 'price', 'range', 'phone', 'extraordinary', 'battery', 'life', 'life', 'easy', 'phone', 'review', 'powerful', 'good', 'work', 'term', 'efficiency', 'heating', 'summer', 'gte', 'phone', 'heating', 'problem', 'last', 'year', 'non', 'existent', 'timedisplay', 'capable', 'dolby', 'vision', 'dolby', 'tech', 'experience', 'immersive', 'price', 'haptic', 'good', 'iphone', 'oxygen', 'good', 'oxygen', 'upgrade', 'year', 'year', 'security', 'update', 'frequent', 'useful', 'phone', 'overall', 'life', 'part', 'window', 'power', 'button', 'old', 'op', 'supportcamera', 'decent', 'iphone', 'low', 'light', 'shot', 'oneplus', 'camera', 'flagship', 'cheap', 'pixel', 'disappointment', 'front', 'camera', 'buy', 'phone', 'vivo', 'oppo', 'approach', 'skin', 'smooth', 'bright', 'front', 'video', 'shame', 'leastcon', 'annoying', 'bugsfeature', 'small', 'updatedisplay', 'bright', 'indoor', 'iphone', 'ultra', 'thoseone', 'apparent', 'disadvantage', 'lack', 'wireless', 'ip', 'rating', 'compr

In [16]:
final_reviews = [' '.join(review) for review in tokenized_reviews]

In [17]:
final_reviews

['oneplus week price range phone extraordinary battery life life easy phone review powerful good work term efficiency heating summer gte phone heating problem last year non existent timedisplay capable dolby vision dolby tech experience immersive price haptic good iphone oxygen good oxygen upgrade year year security update frequent useful phone overall life part window power button old op supportcamera decent iphone low light shot oneplus camera flagship cheap pixel disappointment front camera buy phone vivo oppo approach skin smooth bright front video shame leastcon annoying bugsfeature small updatedisplay bright indoor iphone ultra thoseone apparent disadvantage lack wireless ip rating compromise quad hd display price reverse wireless charging earbud last beautiful thing phone charger screen guard phone case dollar company lakhi helpful oneplus qualcomm year platform year',
 'lot research green line issue life time screen positive move trust customersprosprimary camera task smoothlyg

In [18]:
'''criteria = ["battery", "storage", "cost", "performance", "camera", "quality", "value", "appearance"]

for review in tokenized_reviews:
    for criterion in criteria:
        if criterion not in review:
            review.append(criterion)'''

'criteria = ["battery", "storage", "cost", "performance", "camera", "quality", "value", "appearance"]\n\nfor review in tokenized_reviews:\n    for criterion in criteria:\n        if criterion not in review:\n            review.append(criterion)'

In [19]:
dictionary = corpora.Dictionary(tokenized_reviews)

In [20]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(final_reviews)

In [21]:
print(tfidf_matrix.shape)

(300, 1861)


In [22]:
corpus = gensim.matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)

In [23]:
id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())

In [24]:
LDA = gensim.models.ldamodel.LdaModel

lda_model = LDA(corpus, num_topics=5, random_state=100, chunksize=1000, iterations=500,id2word=id2word,alpha = 'auto',eta='auto')

In [25]:
lda_model.print_topics()

[(0,
  '0.008*"phone" + 0.006*"camera" + 0.006*"good" + 0.005*"battery" + 0.004*"day" + 0.004*"performance" + 0.003*"display" + 0.003*"oneplus" + 0.003*"usage" + 0.003*"month"'),
 (1,
  '0.014*"good" + 0.010*"phone" + 0.008*"oneplus" + 0.006*"camera" + 0.006*"quality" + 0.006*"performance" + 0.004*"bad" + 0.004*"issue" + 0.004*"overall" + 0.004*"photo"'),
 (2,
  '0.009*"phone" + 0.008*"good" + 0.005*"camera" + 0.004*"mobile" + 0.004*"quality" + 0.003*"super" + 0.003*"bad" + 0.003*"fps" + 0.003*"battery" + 0.003*"flagship"'),
 (3,
  '0.016*"good" + 0.008*"quality" + 0.008*"phone" + 0.007*"price" + 0.006*"camera" + 0.006*"great" + 0.005*"premium" + 0.005*"battery" + 0.005*"mobile" + 0.005*"performance"'),
 (4,
  '0.009*"phone" + 0.008*"excellent" + 0.008*"nice" + 0.006*"camera" + 0.005*"good" + 0.005*"battery" + 0.004*"experience" + 0.004*"issue" + 0.004*"display" + 0.004*"iphone"')]

In [26]:

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.008*"phone" + 0.006*"camera" + 0.006*"good" + 0.005*"battery" + 0.004*"day" + 0.004*"performance" + 0.003*"display" + 0.003*"oneplus" + 0.003*"usage" + 0.003*"month"')
(1, '0.014*"good" + 0.010*"phone" + 0.008*"oneplus" + 0.006*"camera" + 0.006*"quality" + 0.006*"performance" + 0.004*"bad" + 0.004*"issue" + 0.004*"overall" + 0.004*"photo"')
(2, '0.009*"phone" + 0.008*"good" + 0.005*"camera" + 0.004*"mobile" + 0.004*"quality" + 0.003*"super" + 0.003*"bad" + 0.003*"fps" + 0.003*"battery" + 0.003*"flagship"')
(3, '0.016*"good" + 0.008*"quality" + 0.008*"phone" + 0.007*"price" + 0.006*"camera" + 0.006*"great" + 0.005*"premium" + 0.005*"battery" + 0.005*"mobile" + 0.005*"performance"')
(4, '0.009*"phone" + 0.008*"excellent" + 0.008*"nice" + 0.006*"camera" + 0.005*"good" + 0.005*"battery" + 0.004*"experience" + 0.004*"issue" + 0.004*"display" + 0.004*"iphone"')


In [27]:
topic_list = []

for topic in topics:
    topic_num, topic_words = topic
    words = [word.split('*')[1].strip('"') for word in topic_words.split(" + ")]
    topic_list.append(words)

topic_lda_Oneplus=[]
[topic_lda_Oneplus.extend(k) for k in topic_list ]
topic_lda_Oneplus

['phone',
 'camera',
 'good',
 'battery',
 'day',
 'performance',
 'display',
 'oneplus',
 'usage',
 'month',
 'good',
 'phone',
 'oneplus',
 'camera',
 'quality',
 'performance',
 'bad',
 'issue',
 'overall',
 'photo',
 'phone',
 'good',
 'camera',
 'mobile',
 'quality',
 'super',
 'bad',
 'fps',
 'battery',
 'flagship',
 'good',
 'quality',
 'phone',
 'price',
 'camera',
 'great',
 'premium',
 'battery',
 'mobile',
 'performance',
 'phone',
 'excellent',
 'nice',
 'camera',
 'good',
 'battery',
 'experience',
 'issue',
 'display',
 'iphone']

In [30]:
def remove_topic(topic_lda_Oneplus):
    t = ['phone', 'oneplus','mobile']
    for i in t:
        while i in topic_lda_Oneplus:
            topic_lda_Oneplus.remove(i)
    return topic_lda_Oneplus

topic_lda_Oneplus = remove_topic(topic_lda_Oneplus)
topic_lda_Oneplus

['camera',
 'good',
 'battery',
 'day',
 'performance',
 'display',
 'usage',
 'month',
 'good',
 'camera',
 'quality',
 'performance',
 'bad',
 'issue',
 'overall',
 'photo',
 'good',
 'camera',
 'quality',
 'super',
 'bad',
 'fps',
 'battery',
 'flagship',
 'good',
 'quality',
 'price',
 'camera',
 'great',
 'premium',
 'battery',
 'performance',
 'excellent',
 'nice',
 'camera',
 'good',
 'battery',
 'experience',
 'issue',
 'display',
 'iphone']

In [29]:
coherence_model_lda = CoherenceModel(
    model=lda_model, 
    texts=tokenized_reviews, 
    dictionary=dictionary, 
    coherence='c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.7103389287889972
